In [4]:
from pathlib import Path

import numpy as np
from PyQt6.QtWidgets import QWidget, QVBoxLayout, QComboBox, QMainWindow, QApplication
from PyQt6.QtCore import pyqtSignal

import matplotlib
matplotlib.use('Qt5Agg')
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg, NavigationToolbar2QT as NavigationToolbar
from matplotlib.figure import Figure



In [ ]:
path_to_session = None

In [5]:

path_to_session = Path(path_to_session)
path_to_freemocap_3d_body_data_npy = path_to_session/'output_data'/'mediapipe_body_3d_xyz.npy'
freemocap_3d_body_data = np.load(path_to_freemocap_3d_body_data_npy)


In [6]:

mediapipe_indices = ['nose',
    'left_eye_inner',
    'left_eye',
    'left_eye_outer',
    'right_eye_inner',
    'right_eye',
    'right_eye_outer',
    'left_ear',
    'right_ear',
    'mouth_left',
    'mouth_right',
    'left_shoulder',
    'right_shoulder',
    'left_elbow',
    'right_elbow',
    'left_wrist',
    'right_wrist',
    'left_pinky',
    'right_pinky',
    'left_index',
    'right_index',
    'left_thumb',
    'right_thumb',
    'left_hip',
    'right_hip',
    'left_knee',
    'right_knee',
    'left_ankle',
    'right_ankle',
    'left_heel',
    'right_heel',
    'left_foot_index',
    'right_foot_index']


In [7]:

class MarkerSelectorWidget(QWidget):
    marker_to_plot_updated_signal = pyqtSignal()
    def __init__(self):
        super().__init__()

        self._layout = QVBoxLayout()
        self.setLayout(self._layout)
        
        combo_box_items = mediapipe_indices
        # combo_box_items.insert(0,'')
        self.marker_combo_box = QComboBox()
        self.marker_combo_box.addItems(combo_box_items)
        self._layout.addWidget(self.marker_combo_box)

        self.current_marker = self.marker_combo_box.currentText()
        self.marker_combo_box.currentTextChanged.connect(self.return_marker)

    def return_marker(self):
        self.current_marker = self.marker_combo_box.currentText()
        self.marker_to_plot_updated_signal.emit()

        return self.current_marker


class TimeSeriesPlotCanvas(FigureCanvasQTAgg):

    def __init__(self, parent=None, width=15, height=4, dpi=100):
        fig = Figure(figsize=(width, height), dpi=dpi)
        self.x_ax = fig.add_subplot(311)
        self.y_ax = fig.add_subplot(312)
        self.z_ax = fig.add_subplot(313)

        super(TimeSeriesPlotCanvas, self).__init__(fig)

class TimeSeriesPlotterWidget(QWidget):

    def __init__(self):
        super().__init__()

        self._layout = QVBoxLayout()
        self.setLayout(self._layout)

        self.fig, self.axes_list = self.initialize_skeleton_plot()

        toolbar = NavigationToolbar(self.fig, self)

        self._layout.addWidget(toolbar)
        self._layout.addWidget(self.fig)

    def initialize_skeleton_plot(self):
        fig = TimeSeriesPlotCanvas(self, width=15, height=10, dpi=100)
        self.x_ax = fig.figure.axes[0]
        self.y_ax = fig.figure.axes[1]
        self.z_ax = fig.figure.axes[2]

        self.axes_list = [self.x_ax,self.y_ax,self.z_ax]
        return fig, self.axes_list

    def get_mediapipe_indices(self,marker_to_plot):
        mediapipe_index = mediapipe_indices.index(marker_to_plot)
        return mediapipe_index
    

    def update_plot(self,marker_to_plot:str, freemocap_data:np.ndarray):
        mediapipe_index = self.get_mediapipe_indices(marker_to_plot)

        axes_names = ['X Axis', 'Y Axis', 'Z Axis']

        for dimension, (ax,ax_name) in enumerate(zip(self.axes_list,axes_names)):

            ax.cla()
            ax.plot(freemocap_data[:,mediapipe_index,dimension], label = 'FreeMoCap', alpha = .7)

            ax.set_ylabel(ax_name)
            
            if dimension == 2: #put the xlabel only on the last plot
                ax.set_xlabel('Frame #')

            ax.legend()

        self.fig.figure.canvas.draw_idle()


class TimeSeriesViewer(QWidget):
    def __init__(self, freemocap_data:np.ndarray):
        super().__init__()

        self.layout = QVBoxLayout()
        self.setLayout(self.layout)


        self.freemocap_data = freemocap_data

        self.marker_selector_widget = MarkerSelectorWidget()
        self.layout.addWidget(self.marker_selector_widget)

        self.time_series_plotter_widget = TimeSeriesPlotterWidget()
        self.layout.addWidget(self.time_series_plotter_widget)


        self.connect_signals_to_slots()

    def connect_signals_to_slots(self):
        self.marker_selector_widget.marker_to_plot_updated_signal.connect(lambda: self.time_series_plotter_widget.update_plot(self.marker_selector_widget.current_marker,self.freemocap_data))


In [8]:

class TimeSeriesPlotCanvas(FigureCanvasQTAgg):

    def __init__(self, parent=None, width=15, height=4, dpi=100):
        fig = Figure(figsize=(width, height), dpi=dpi)
        self.x_ax = fig.add_subplot(311)
        self.y_ax = fig.add_subplot(312)
        self.z_ax = fig.add_subplot(313)

        super(TimeSeriesPlotCanvas, self).__init__(fig)

class TimeSeriesPlotterWidget(QWidget):

    def __init__(self):
        super().__init__()

        self._layout = QVBoxLayout()
        self.setLayout(self._layout)

        self.fig, self.axes_list = self.initialize_skeleton_plot()

        toolbar = NavigationToolbar(self.fig, self)

        self._layout.addWidget(toolbar)
        self._layout.addWidget(self.fig)

    def initialize_skeleton_plot(self):
        fig = TimeSeriesPlotCanvas(self, width=15, height=10, dpi=100)
        self.x_ax = fig.figure.axes[0]
        self.y_ax = fig.figure.axes[1]
        self.z_ax = fig.figure.axes[2]

        self.axes_list = [self.x_ax,self.y_ax,self.z_ax]
        return fig, self.axes_list

    def get_mediapipe_indices(self,marker_to_plot):
        mediapipe_index = mediapipe_indices.index(marker_to_plot)
        return mediapipe_index
    

    def update_plot(self,marker_to_plot:str, freemocap_data:np.ndarray):
        mediapipe_index = self.get_mediapipe_indices(marker_to_plot)

        axes_names = ['X Axis', 'Y Axis', 'Z Axis']

        for dimension, (ax,ax_name) in enumerate(zip(self.axes_list,axes_names)):

            ax.cla()
            ax.plot(freemocap_data[:,mediapipe_index,dimension], label = 'FreeMoCap', alpha = .7)

            ax.set_ylabel(ax_name)
            
            if dimension == 2: #put the xlabel only on the last plot
                ax.set_xlabel('Frame #')

            ax.legend()

        self.fig.figure.canvas.draw_idle()


In [9]:
class TimeSeriesViewer(QWidget):
    def __init__(self, freemocap_data:np.ndarray):
        super().__init__()

        self.layout = QVBoxLayout()
        self.setLayout(self.layout)


        self.freemocap_data = freemocap_data

        self.marker_selector_widget = MarkerSelectorWidget()
        self.layout.addWidget(self.marker_selector_widget)

        self.time_series_plotter_widget = TimeSeriesPlotterWidget()
        self.layout.addWidget(self.time_series_plotter_widget)


        self.connect_signals_to_slots()

    def connect_signals_to_slots(self):
        self.marker_selector_widget.marker_to_plot_updated_signal.connect(lambda: self.time_series_plotter_widget.update_plot(self.marker_selector_widget.current_marker,self.freemocap_data))


In [10]:
class MarkerTimeSeriesWindow(QMainWindow):
    def __init__(self, freemocap_data:np.ndarray):
        super().__init__()
    
        layout = QVBoxLayout()
        widget = QWidget()

        self.time_series_viewer = TimeSeriesViewer(freemocap_data)
        layout.addWidget(self.time_series_viewer)

        widget.setLayout(layout)
        self.setCentralWidget(widget)




app = QApplication([])
win = MarkerTimeSeriesWindow(freemocap_3d_body_data)
win.show()
app.exec()